In [ ]:
!pip install selenium

In [ ]:
!pip install webdriver-manager

In [ ]:
!pip install --upgrade pip


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from urllib import error
from urllib import request
import os
import time
import sys

# default url
# replace for yours
url = "https://www.bing.com/?scope=images&nr=1&FORM=NOFORM"
explorer = "Safari"
# directory
imgs_dir = "./images/"


# report hook with three parameters passed
# count_of_blocks  The number of blocks transferred
# block_size The size of block
# total_size Total size of the file
def progress_callback(count_of_blocks, block_size, total_size):
    # determine current progress
    progress = int(50 * (count_of_blocks * block_size) / total_size)
    if progress > 50:
        progress = 50
    # update progress bar
    sys.stdout.write("\r[%s%s] %d%%" % ('█' * progress, '  ' * (50 - progress), progress * 2))
    sys.stdout.flush()


class CrawlSelenium:

	def __init__(self, keyword, explorer="Safari", url="https://www.google.com"):
		self.url = url
		self.explorer = explorer
		self.keyword = keyword

	def set_loading_strategy(self, strategy="normal"):
		self.options = Options()
		self.options.page_load_strategy = strategy


	def crawl(self):
		# instantiate driver according to corresponding explorer
		if self.explorer == "Chrome":
			self.driver = webdriver.Chrome(options=self.options)
		if self.explorer == "Opera":
			self.driver = webdriver.Opera(options=self.options)
		if self.explorer == "Firefox":
			self.driver = webdriver.Firefox(options=self.options)
		if self.explorer == "Edge":
			self.driver = webdriver.Edge(options=self.options)
		if self.explorer == "Safari":
			self.driver = webdriver.Safari(options=self.options)

		# search on google
		# navigate to url
		self.driver.get(self.url)
		# locate input field
		search_input = self.driver.find_element(By.NAME, 'q')
		# emulate user input and enter to search
		webdriver.ActionChains(self.driver).move_to_element(search_input).send_keys(self.keyword + Keys.ENTER).perform()
		
		# self.driver.find_element(By.LINK_TEXT, '我同意').click()
		# # navigate to google image
		# # find navigation buttons
		# self.driver.find_element(By.LINK_TEXT, '图片').click()

		# load more images as many as possible
		# scrolling to bottom
		self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
		# get button
		# show_more_button = self.driver.find_element(By.CSS_SELECTOR, "input[value='显示更多搜索结果']")
		# try:
		# 	while True:
		# 		# do according to message
		# 		message = self.driver.find_element(By.CSS_SELECTOR, 'div.OuJzKb.Bqq24e').get_attribute('textContent')
		# 		# print(message)
		# 		if message == '正在加载更多内容，请稍候':
		# 			self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
		# 		elif message == '新内容已成功加载。向下滚动即可查看更多内容。':
		# 			# scrolling to bottom
		# 			self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
		# 			if show_more_button.is_displayed():
		# 				show_more_button.click()
		# 		elif message == '看来您已经看完了所有内容':
		# 			break
		# 		elif message == '无法加载更多内容，点击即可重试。':
		# 			show_more_button.click()
		# 		else:
		# 			self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
		# except Exception as err:
		# 	print(err)
		message = self.driver.find_element(By.CSS_SELECTOR, 'div.OuJzKb.Bqq24e').get_attribute('textContent')
		print(message)
		# find all image elements in google image result page
		imgs = self.driver.find_elements(By.CSS_SELECTOR, "img.rg_i.Q4LuWd")
		
		img_count = 0
		for img in imgs:
			try:
				# image per second
				time.sleep(1)
				
				img_url = img.get_attribute("src")
				if img_url == None:
					continue
				path = os.path.join(imgs_dir,keyword,'/',str(img_count) + "_img.jpg")
				request.urlretrieve(url = img_url, filename = path, reporthook = progress_callback, data = None)
				print('\ndownloaded image ' + str(img_count) + ': ')
				img_count = img_count + 1
				if(img_count>2): break
			except error.HTTPError as http_err:
				print(http_err)
			except Exception as err:
				print(err)


def main():
	# setting
	keyword = 'Hera Pheri' + ' film imdb'
	crawl_s = CrawlSelenium(keyword, explorer, url)
	crawl_s.set_loading_strategy("normal")
	# make directory
	if not os.path.exists(imgs_dir):
		os.mkdir(imgs_dir)
	# crawling
	crawl_s.crawl()


if __name__ == "__main__":
	main()

In [ ]:
from selenium import webdriver
import time
import os
import requests
from PIL import Image
from io import BytesIO




class Crawler_google_images:
    # 初始化
    def __init__(self, keyword, url):
        self.url = url
        self.keyword = keyword

    # 获得Chrome驱动，并访问url
    def init_browser(self):
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-infobars")
        browser = webdriver.Chrome(chrome_options=chrome_options)
        # 访问url
        browser.get(self.url)
        # 最大化窗口，之后需要爬取窗口中所见的所有图片
        browser.maximize_window()
        return browser

    #下载图片
    def download_images(self, browser,round=2):
        picpath = './images/'+keyword
        # 路径不存在时创建一个
        if not os.path.exists(picpath): os.makedirs(picpath)
        # 记录下载过的图片地址，避免重复下载
        img_url_dic = []

        count = 0 #图片序号
        pos = 0
        for i in range(round):
            pos += 500
            # 向下滑动
            js = 'var q=document.documentElement.scrollTop=' + str(pos)
            browser.execute_script(js)
            time.sleep(1)
            # 找到图片
            # html = browser.page_source#也可以抓取当前页面的html文本，然后用beautifulsoup来抓取
            #直接通过tag_name来抓取是最简单的，比较方便

            img_elements = browser.find_elements_by_tag_name('img')
            #遍历抓到的webElement
            for img_element in img_elements:
                img_url = img_element.get_attribute('src')
                # print(img_url)
                # 前几个图片的url太长，不是图片的url，先过滤掉，爬后面的
                if isinstance(img_url, str):
                    if len(img_url) <= 200:
                        #将干扰的goole图标筛去
                        if 'images' in img_url:
                            #判断是否已经爬过，因为每次爬取当前窗口，或许会重复
                            #实际上这里可以修改一下，将列表只存储上一次的url，这样可以节省开销，不过我懒得写了···
                            if img_url not in img_url_dic:
                                try:
                                    img_url_dic.append(img_url)
                                    #下载并保存图片到当前目录下
                                    filename = picpath+'/'+ str(count) + ".jpg"
                                    r = requests.get(img_url)
                                    # print("r: ", r)
                                    # print("Trying to open the image")
                                    img = Image.open(BytesIO(r.content))
                                    width, height = img.size
                                    print("width: ", width , ", height: ", height)
                                    if(height>width):
                                        with open(filename, 'wb') as f:
                                            f.write(r.content)
                                        f.close()
                                        count += 1
                                        print('this is '+'img '+str(count))
                                    
                                    #防止反爬机制
                                    time.sleep(0.2)
                                except:
                                    print('failure')
                if(count>2): break

    def run(self):
        browser = self.init_browser()
        self.download_images(browser,10)#可以修改爬取的页面数，基本10页是100多张图片
        browser.close()
        print("爬取完成")


if __name__ == '__main__':
    keyword = 'Your Name.' + ' imdb poster'
    url = 'https://www.google.com.hk/search?q='+keyword+'&tbm=isch'
    print(url)
    craw = Crawler_google_images(keyword, url)
    craw.run()

In [ ]:
from selenium import webdriver
import time
import os
import re
import requests
from PIL import Image
from io import BytesIO
from selenium.webdriver.common.by import By



class Crawler_google_images:
    # 初始化
    def __init__(self, keyword, url):
        self.url = url
        self.keyword = keyword

    # 获得Chrome驱动，并访问url
    def init_browser(self):
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-infobars")
        browser = webdriver.Chrome(chrome_options=chrome_options)
        # 访问url
        browser.get(self.url)
        # 最大化窗口，之后需要爬取窗口中所见的所有图片
        browser.maximize_window()
        return browser

    #下载图片
    def download_images(self, browser,round=2):
        picpath = './images/'+keyword
        # 路径不存在时创建一个
        if not os.path.exists(picpath): os.makedirs(picpath)
        # 记录下载过的图片地址，避免重复下载
        img_url_dic = []

        count = 0 #图片序号
        pos = 0
        for i in range(round):
            pos += 500
            # 向下滑动
            js = 'var q=document.documentElement.scrollTop=' + str(pos)
            browser.execute_script(js)
            time.sleep(1)
            # 找到图片
            # html = browser.page_source#也可以抓取当前页面的html文本，然后用beautifulsoup来抓取
            #直接通过tag_name来抓取是最简单的，比较方便

            img_elements = browser.find_elements(By.CLASS_NAME, 'iusc')
            # print(img_elements)
            for img_element in img_elements:
                m = img_element.get_attribute('m')
                print(m)
                img_url = re.findall("murl\":\"https?://[^\s]*.jpg", m)
                print(img_url[0][7:])
            #遍历抓到的webElement
            # for img_element in img_elements:
            #     img_url = img_element.get_attribute('src')
            #     # print(img_url)
            #     # 前几个图片的url太长，不是图片的url，先过滤掉，爬后面的
            #     if isinstance(img_url, str):
            #         if len(img_url) <= 200:
            #             #将干扰的goole图标筛去
            #             if 'images' in img_url:
            #                 #判断是否已经爬过，因为每次爬取当前窗口，或许会重复
            #                 #实际上这里可以修改一下，将列表只存储上一次的url，这样可以节省开销，不过我懒得写了···
            #                 if img_url not in img_url_dic:
            #                     try:
            #                         img_url_dic.append(img_url)
            #                         #下载并保存图片到当前目录下
            #                         filename = picpath+'/'+ str(count) + ".jpg"
            #                         r = requests.get(img_url)
            #                         # print("r: ", r)
            #                         # print("Trying to open the image")
            #                         img = Image.open(BytesIO(r.content))
            #                         width, height = img.size
            #                         print("width: ", width , ", height: ", height)
            #                         if(height>width):
            #                             with open(filename, 'wb') as f:
            #                                 f.write(r.content)
            #                             f.close()
            #                             count += 1
            #                             print('this is '+'img '+str(count))
                                    
            #                         #防止反爬机制
            #                         time.sleep(0.2)
            #                     except:
            #                         print('failure')
            #     if(count>2): break

    def run(self):
        browser = self.init_browser()
        self.download_images(browser,10)#可以修改爬取的页面数，基本10页是100多张图片
        browser.close()
        print("爬取完成")


if __name__ == '__main__':
    keyword = 'Your Name.' + ' film poster'
    url = 'https://www.bing.com/images/search?q='+keyword+'&tbm=isch'
    print(url)
    craw = Crawler_google_images(keyword, url)
    craw.run()

In [ ]:
<a class="iusc" 
style="height:279px;width:186px" 
m="{&quot;sid&quot;:&quot;&quot;,&quot;cturl&quot;:&quot;&quot;,&quot;cid&quot;:&quot;u7VulGqy&quot;,&quot;purl&quot;:&quot;https://www.alternateending.com/2017/04/your-name-2016.html&quot;,&quot;murl&quot;:&quot;https://www.alternateending.com/wp-content/uploads/2017/04/l9KxbI1LaCXtuEOtZe8P2uNOeAR-1024x1536.jpg&quot;,&quot;turl&quot;:&quot;https://tse2.mm.bing.net/th?id=OIP.u7VulGqy2EbnU4Htk3GwYAHaLH&amp;pid=15.1&quot;,&quot;md5&quot;:&quot;bbb56e946ab2d846e75381ed9371b060&quot;,&quot;shkey&quot;:&quot;b2X7CY8MNiHQ565QOTRWwUAIQAT9UBEuM5DVY7IYChQ=&quot;,&quot;t&quot;:&quot;Your Name. (2016) - Movie Review : Alternate Ending&quot;,&quot;mid&quot;:&quot;8127F546E86BE3F755AF4963D96FD40CBA65AD31&quot;,&quot;desc&quot;:&quot;yourname shinkai makoto alternateending sinopse&quot;}" 
onclick="sj_evt.fire('IFrame.Navigate', this.href); return false;" 
href="/images/search?view=detailV2&amp;ccid=u7VulGqy&amp;id=8127F546E86BE3F755AF4963D96FD40CBA65AD31&amp;thid=OIP.u7VulGqy2EbnU4Htk3GwYAHaLH&amp;mediaurl=https%3a%2f%2fwww.alternateending.com%2fwp-content%2fuploads%2f2017%2f04%2fl9KxbI1LaCXtuEOtZe8P2uNOeAR-1024x1536.jpg&amp;cdnurl=https%3a%2f%2fth.bing.com%2fth%2fid%2fR.bbb56e946ab2d846e75381ed9371b060%3frik%3dMa1lugzUb9ljSQ%26pid%3dImgRaw%26r%3d0&amp;exph=1536&amp;expw=1024&amp;q=Your+Name.+film+poster&amp;simid=607989334476718169&amp;FORM=IRPRST&amp;ck=4F31AC3C20993CB869B9952E4E55CA67&amp;selectedIndex=0" 
h="ID=images,5183.1" 
data-focevt="1">

https://www.alternateending.com/wp-content/uploads/2017/04/l9KxbI1LaCXtuEOtZe8P2uNOeAR-1024x1536.jpg

In [ ]:
! pip install Pillow

In [ ]:
# date：2020.5.25
# author:pmy
# aim:爬取google图片
#问题在于，不能保证所爬为所见

from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
import requests

class Crawler_google_images:
    # 初始化
    def __init__(self, keyword, url):
        self.url = url
        self.keyword = keyword

    # 获得Chrome驱动，并访问url
    def init_browser(self):
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-infobars")
        browser = webdriver.Chrome(chrome_options=chrome_options)
        # 访问url
        browser.get(self.url)
        # 最大化窗口，之后需要爬取窗口中所见的所有图片
        browser.maximize_window()
        return browser

    # 下载图片
    def download_images(self, browser, num=100):
        #存储路径
        picpath = './images/'+keyword
        # 路径不存在时创建一个
        if not os.path.exists(picpath): os.makedirs(picpath)

        count = 0  # 图片序号
        pos = 0
        # print(num)

        while (True):
            # try:
            #     # 向下滑动
            #     js = 'var q=document.documentElement.scrollTop=' + str(pos)
            #     pos += 10
            #     browser.execute_script(js)
            #     time.sleep(1)
            #     # 找到图片
            #     # html = browser.page_source#也可以抓取当前页面的html文本，然后用beautifulsoup来抓取
            #     # 直接通过tag_name来抓取是最简单的，比较方便
            # except:
            #     print("划不动了")
            img_elements = browser.find_elements(By.XPATH, '//a[@class="wXeWr islib nfEiy mM5pbd"]')
            try:
                for img_element in img_elements:
                    #点开大图页面
                    img_element.click()
                    time.sleep(0.5)
                    try:
                        # 这里balabala里面有好几个，所以要过滤一下
                        # 取名好烦哦···
                        balabalas = browser.find_elements(By.XPATH,'//img[@class="n3VNCb"]')

                        if (balabalas):
                            for balabala in balabalas:
                                src = balabala.get_attribute('src')
                                #过滤掉缩略图和无关干扰信息
                                if src.startswith('http') and not src.startswith(
                                        'https://encrypted-tbn0.gstatic.com'):
                                    print('Found' + str(count) + 'st image url')
                                    # img_url_dic.append(src)
                                    self.save_img(count, src, picpath)
                                    count += 1
                                    #爬取到指定数量图片后退出
                                    if (count >= num):
                                        return "stop"
                    except:
                        print('获取图片失败')
                #回退
                browser.back()
                time.sleep(0.3)
            except:
                print('获取页面失败')

    def save_img(self, count, img_src, picpath):
        filename = picpath + '/' + str(count) + '.jpg'
        r = requests.get(img_src)
        with open(filename, 'wb') as f:
            f.write(r.content)
        f.close()

    def run(self):
        browser = self.init_browser()
        self.download_images(browser, 10)  # 可以修改爬取的图片数
        browser.close()
        print("############爬取完成")


if __name__ == '__main__':
    keyword = 'Hera Pheri' + ' film imdb poster'
    url = 'https://www.google.com.hk/search?q='+keyword+'&tbm=isch'
    craw = Crawler_google_images(keyword, url)
    craw.run()




In [5]:
from selenium import webdriver
import time
import os
import re
import requests
from PIL import Image
from io import BytesIO
from selenium.webdriver.common.by import By



class Crawler_google_images:
    # 初始化
    def __init__(self, keyword, url):
        self.url = url
        self.keyword = keyword

    # 获得Chrome驱动，并访问url
    def init_browser(self):
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--disable-infobars")
        browser = webdriver.Chrome(chrome_options=chrome_options)
        # 访问url
        browser.get(self.url)
        # 最大化窗口，之后需要爬取窗口中所见的所有图片
        browser.maximize_window()
        return browser

    #下载图片
    def download_images(self, browser,round=2):
        picpath = './images/'+keyword
        # 路径不存在时创建一个
        if not os.path.exists(picpath): os.makedirs(picpath)

        count = 0 #图片序号
        pos = 0
        for i in range(round):
            pos += 500
            # 向下滑动
            js = 'var q=document.documentElement.scrollTop=' + str(pos)
            browser.execute_script(js)
            time.sleep(1)
            # 找到图片
            # html = browser.page_source#也可以抓取当前页面的html文本，然后用beautifulsoup来抓取
            #直接通过tag_name来抓取是最简单的，比较方便

            img_elements = browser.find_elements(By.CLASS_NAME, 'iusc')
            # print(img_elements)
            for img_element in img_elements:
                m = img_element.get_attribute('m')
                # print(m)
                img_url = re.findall("murl\":\"https?://[^\s]*.jpg", m)
                # print(img_url[0][7:])
                filename = picpath+'/'+ str(count) + ".jpg"
                r = requests.get(img_url[0][7:])
                img = Image.open(BytesIO(r.content))
                width, height = img.size
                print("width: ", width , ", height: ", height)
                if(height>width):
                    with open(filename, 'wb') as f:
                        f.write(r.content)
                    f.close()
                    count += 1
                    print('this is '+'img '+str(count))
                #防止反爬机制
                time.sleep(0.2)
                if(count>2): break
            if(count>2): break
    def run(self):
        browser = self.init_browser()
        self.download_images(browser,10)#可以修改爬取的页面数，基本10页是100多张图片
        browser.close()
        print("爬取完成")


if __name__ == '__main__':
    keyword = 'Your Name.' + ' film poster'
    url = 'https://www.bing.com/images/search?q='+keyword+'&tbm=isch'
    print(url)
    craw = Crawler_google_images(keyword, url)
    craw.run()

https://www.bing.com/images/search?q=Your Name. film poster&tbm=isch


<ipython-input-5-35a4b4ef0584>:22: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(chrome_options=chrome_options)


width:  1024 , height:  1536
this is img 1
width:  474 , height:  712
this is img 2
width:  1600 , height:  2400
this is img 3
爬取完成
